# Midterm 422 Zhe Wang

In [18]:
import requests
import time
import re
import os
import pandas as pd
import urllib.parse
import mysql.connector
import pymysql
import warnings
import json
from pandas.io.json import json_normalize
from pprintpp import pprint as pp
from types import SimpleNamespace
from bs4 import BeautifulSoup
from os import listdir
from os.path import isfile, join
from tqdm import trange
from tqdm import notebook
import tqdm
from sqlalchemy import create_engine
from requests import get
from time import sleep

## Question 1

In [25]:
# 1(a)
# Install and run SQL on your machine. Use some GUI to see / test that it runs.

# 1(b)
# Store IP Address as VARCHAR, city as VARCHAR, and zip as INT
# The reason to store ip address as varchar is that ip address doesn't have a fixed length and contains periods and numbers
# The reason to store city as varchar is that city doesn't have a fixed length and made up with characters
#The reason to store zip as character is that zip is made up by fixed length of numbers which we usually treated as character.

# 1(c)
warnings.filterwarnings("ignore")
SQL_DB = "msba"
TERM = "blog"

def main():
    
    SQL_TABLE = "ip_addresses"
    SQL_TABLE_DEF = "(" + \
            "ip VARCHAR(50)" + \
            ",city VARCHAR(100)" + \
            ",zip CHAR" + \
            ")"

    create_sql_table(SQL_TABLE, SQL_TABLE_DEF)

def create_sql_table(SQL_TABLE, SQL_TABLE_DEF):
    try:      
        #connect to server
        conn = pymysql.connect(host='localhost', user = 'root', password = '')
        cursor = conn.cursor()

        query = "CREATE DATABASE IF NOT EXISTS " + SQL_DB
        print(query)
        cursor.execute(query);
        
        query = "CREATE TABLE IF NOT EXISTS " + SQL_DB + "." + SQL_TABLE + " " + SQL_TABLE_DEF + ";";
        print(query)
        cursor.execute(query);
        cursor.close()
        conn.close()
        return

    except IOError as e:
        print(e)

if __name__ == "__main__":
    main()

CREATE DATABASE IF NOT EXISTS msba
CREATE TABLE IF NOT EXISTS msba.ip_addresses (ip VARCHAR(50),city VARCHAR(100),zip INT);


## Question 2

In [ ]:
# 2(a)
# API Key = 07ce86386941e835036aebe8d4a2e78a

In [ ]:
# 2(b)
# URL strings:
# http://api.ipstack.com/8.8.8.8?access_key=07ce86386941e835036aebe8d4a2e78a&fields=main;
# http://api.ipstack.com/128.120.0.25?access_key=07ce86386941e835036aebe8d4a2e78a&fields=main;
# http://api.ipstack.com/128.32.12.14?access_key=07ce86386941e835036aebe8d4a2e78a&fields=main;
# http://api.ipstack.com/64.165.72.144?access_key=07ce86386941e835036aebe8d4a2e78a&fields=main;
# http://api.ipstack.com/108.204.6.219?access_key=07ce86386941e835036aebe8d4a2e78a&fields=main;

In [94]:
# 2(c)
# Execute the five API calls
resp1 = requests.get("http://api.ipstack.com/8.8.8.8?access_key=07ce86386941e835036aebe8d4a2e78a&fields=main").json()
resp2 = requests.get("http://api.ipstack.com/128.120.0.25?access_key=07ce86386941e835036aebe8d4a2e78a&fields=main").json()
resp3 = requests.get("http://api.ipstack.com/128.32.12.14?access_key=07ce86386941e835036aebe8d4a2e78a&fields=main").json()
resp4 = requests.get("http://api.ipstack.com/64.165.72.144?access_key=07ce86386941e835036aebe8d4a2e78a&fields=main").json()
resp5 = requests.get("http://api.ipstack.com/108.204.6.219?access_key=07ce86386941e835036aebe8d4a2e78a&fields=main").json()

for i in range(1,6):
    pprint(eval('resp'+str(i)))

{
    'city': 'Mountain View',
    'continent_code': 'NA',
    'continent_name': 'North America',
    'country_code': 'US',
    'country_name': 'United States',
    'ip': '8.8.8.8',
    'latitude': 37.38801956176758,
    'longitude': -122.07431030273438,
    'region_code': 'CA',
    'region_name': 'California',
    'type': 'ipv4',
    'zip': '94041',
}
{
    'city': 'Davis',
    'continent_code': 'NA',
    'continent_name': 'North America',
    'country_code': 'US',
    'country_name': 'United States',
    'ip': '128.120.0.25',
    'latitude': 38.56296157836914,
    'longitude': -121.81600952148438,
    'region_code': 'CA',
    'region_name': 'California',
    'type': 'ipv4',
    'zip': '95616',
}
{
    'city': 'Berkeley',
    'continent_code': 'NA',
    'continent_name': 'North America',
    'country_code': 'US',
    'country_name': 'United States',
    'ip': '128.32.12.14',
    'latitude': 37.87459945678711,
    'longitude': -122.25466918945312,
    'region_code': 'CA',
    'region_n

In [29]:
#2(d)
# Parse the json strings
new_dic = dict()
for i in range(1,6):
    new_dic[i] = json.loads(json.dumps(eval('resp'+str(i))), object_hook=lambda d: SimpleNamespace(**d))
    print('city:', new_dic[i].city)
    print('zip:', new_dic[i].zip)

city: Mountain View
zip: 94041
city: Davis
zip: 95616
city: Berkeley
zip: 94720
city: West Sacramento
zip: 95819
city: San Bruno
zip: 94066


In [33]:
#2(e)
# Parse the json strings
new_dic = dict()
for i in range(1,6):
    new_dic[i] = json.loads(json.dumps(eval('resp'+str(i))), object_hook=lambda d: SimpleNamespace(**d))
    print('ip:', new_dic[i].ip)
    print('city:',new_dic[i].city)
    print('zip:', new_dic[i].zip)

ip: 8.8.8.8
city: Mountain View
zip: 94041
ip: 128.120.0.25
city: Davis
zip: 95616
ip: 128.32.12.14
city: Berkeley
zip: 94720
ip: 64.165.72.144
city: West Sacramento
zip: 95819
ip: 108.204.6.219
city: San Bruno
zip: 94066


In [36]:
# Add data to database msba
msba = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database="msba"
)

mycursor = msba.cursor()

sql = "INSERT INTO ip_addresses (ip, city, zip) VALUES (%s, %s, %s)"
val = [
    ("8.8.8.8", "Mountain View", 94041),
    ("128.120.0.25", "Davis", 95616),
    ("128.32.12.14", "Berkeley", 94720),
    ("64.165.72.144", "West Sacramento", 95819),
    ("108.204.6.219", "San Bruno", 94066)
]
mycursor.executemany(sql, val)
msba.commit()

5 record inserted.


# Question 3

In [3]:
# 3(b)
for x in range(1,9):
    #Create headers
    headers = {'User-agent': 'Mozilla/5.0'} 
    URL = "https://www.ebay.com/e/daily-deals/hiw-presidents-day-deals-white-sale?_pgn=" + str(x)
    page = requests.get(URL, headers=headers)
    name = 'deal'+ str(x) +'.htm'
    #Sleep 10 seconds between each visit
    time.sleep(10)
    #Save .htm files
    with open(name,'w', encoding='utf-8') as file:
        file.write(page.text)

In [4]:
list_ = []
#Loop through all htm files and look for all item links
for x in range(1,9):
    name = 'deal'+str(x)+'.htm'
    with open(name, 'r', encoding='utf-8') as file:
        text = file.read()
        soup = BeautifulSoup(text, 'html.parser')
        div = soup.find_all("div", class_= "s-item__info clearfix")
        for i in div:
            links = i.select('a', class_= 's-item__link')
            url = links[0].get('href')
            #Here we insert all links to a list
            list_.append(url)
            with open("deals.txt", "w") as output:
                for s in list_:
                    output.write("%s\n" % s)

In [5]:
#Total number of links
len(list_)

184

In [13]:
# 3(c)
#Download every item page to a folder
path = os.getcwd() + "/deals"
try:
    os.mkdir(path)
except:
    print('path already exist')
with open("deals.txt", "r", encoding='utf-8') as file:
    file = file.read().split("\n")
    for x in trange(0, len(list_)):
        headers = {'User-agent': 'Mozilla/5.0'}
        URL = file[x]
        page = requests.get(URL, headers=headers)
        name = str(URL.split('/', 5)[-1].split('?')[0]) + '.htm'
        #Sleep between each access to item pages
        time.sleep(4)
        with open(os.path.join(path, '') + name,'w', encoding='utf-8') as f:
            f.write(page.text)

  0%|          | 0/184 [00:00<?, ?it/s]

path already exist


100%|██████████| 184/184 [13:52<00:00,  4.52s/it]


In [14]:
# 3(d)
# Functions that extract useful information
# Get title for each item
def title(soup):
    #Find title of items by h1 tag
    title = soup.find("h1", class_='it-ttl')
    title_1 = title.get_text().split('  ')[-1][1:] 
    return title_1

# Get selling number for each item
def number_sold(soup):
    #Find number sold by a tag
    sold = soup.find("a", class_="vi-txt-underline")
    try:
        sold_1 = sold.get_text().split(' ')[0]
    except:
        return None
    else:
        sold_2 = int(sold_1.replace(',',''))
        return sold_2
    
# Get whether this item can be returned
def return_allowed(soup):
    #Find whether item can be returned by span tag
    return_ = soup.find("span", id= "vi-ret-accrd-txt")
    for i in return_.text:
        check = i.isdigit()
        if check==True:
            break
    return check

# Get the condition status of each item
def condition_status(soup):
    #Find condition status by span tag with id
    condition_status = soup.find("span", id="vi-cond-addl-info")
    condition_status_1 = condition_status.previous_sibling[:-2].split('\t')[-1]
    return condition_status_1

# Get the first shipping price for each item
def shipping_price(soup):
    #Here extract shipping price. If there is only free shipping option, then the shipping price = 0
    shipping_price = soup.find('span', id='shSummary').contents[1].text.split(' ')[-1]
    if 'FREE' in shipping_price.upper():
        return 0
    else:
        return shipping_price
    
# Get the seller's name for each item
def seller_name(soup):
    #Use span tage with class mbg-nw to locate the text of seller's name
    seller_name = soup.find_all('span', {'class': 'mbg-nw'})[0].get_text()
    return seller_name

# Get seller's score for each item
def seller_score(soup):
    #Use span tag with class mbg-1 to location seller's score. Use regular expression to elimilate blank spaces
    seller_score = re.sub(r'[^\d]', '', soup.find_all('span', {'class': 'mbg-l'})[0].get_text())
    return int(seller_score)

# Get the item price. It can be either regular price or unit price.
def item_price(soup):
    item_price = None
    for trial in (
        # Here to get regular price
        lambda: re.sub(r'[^\d.]', '', soup.find(id='vi-mskumap-none').get_text()),
        # Here to get unit price
        lambda: re.sub(r'[^\d.]', '', soup.find('span', class_='notranslate').get_text())
    ):
        try:
            item_price = trial()
        except:
            continue
        break    
    else:
        print('Warning: No Items Price Extracted')
    try:
        item_price = int(float(item_price) * 100)
    except:
        item_price
        
    return item_price 

# Get list prices for each item. Some items don't have list prices
def list_price(soup):
    try:
        list_price = re.sub(r'[^\d.]', '', soup.find('span', class_= 'notranslate ms-orp').get_text())
    except:
        return None
    else:
        return int(float(list_price) * 100)

In [15]:
# This is the main operational function. I created a dataframe to hold information for each item
def get_information(mypath):
    files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    items = pd.DataFrame(files, columns=['item_id'])
    items['item_id'] = items['item_id'].str.replace('.htm', '')
    #Loop through each item and fill their information in the data frame
    for file_name in tqdm.notebook.tqdm(files):
        item_id = re.sub(r'[^\d]', '', file_name)
        with open(file_name, encoding="utf8") as open_file:
            # parse the page and find appropriate tag
            soup = BeautifulSoup(open_file, 'html.parser')
            items.loc[items['item_id'] == item_id, 'seller_score'] = seller_score(soup)
            items.loc[items['item_id'] == item_id, 'item_price'] = item_price(soup)          
            items.loc[items['item_id'] == item_id, 'list_price'] = list_price(soup)
            
            # Here we insert this column so it can be used in the follwoing question
            if list_price(soup) == None:
                items.loc[items['item_id'] == item_id, 'has_list_price'] = False
            else:
                items.loc[items['item_id'] == item_id, 'has_list_price'] = True           
            items.loc[items['item_id'] == item_id, 'title'] = title(soup)       
            items.loc[items['item_id'] == item_id, 'number_sold'] = number_sold(soup)       
            items.loc[items['item_id'] == item_id, 'return_allowed'] = return_allowed(soup)      
            items.loc[items['item_id'] == item_id, 'condition_status'] = condition_status(soup)
            items.loc[items['item_id'] == item_id, 'seller_name'] = seller_name(soup)   
            items.loc[items['item_id'] == item_id, 'shipping_price'] = shipping_price(soup)

    return items

In [19]:
os.chdir(path)
items = get_information(path)
# Count total NaN at each column 
print(" \nCount total NaN at each column : \n\n", 
      items.isnull().sum()) 
items.head()


 
Count total NaN at each column : 

 item_id              0
seller_score         0
item_price           0
list_price          98
has_list_price       0
title                0
number_sold          8
return_allowed       0
condition_status     0
seller_name          0
shipping_price       0
dtype: int64


,item_id,seller_score,item_price,list_price,has_list_price,title,number_sold,return_allowed,condition_status,seller_name,shipping_price
0,110841365290,219961.0,799.0,1698.0,True,SET OF 1500 TC PILLOWCASES TWO PILLOW CASES PE...,2795.0,True,New with tags,cozyarray,0.0
1,110924201040,219961.0,799.0,1798.0,True,1200 SERIES PILLOWCASES - 2 Pillow Cases Per S...,14867.0,True,New with tags,cozyarray,0.0
2,111634181825,219961.0,2199.0,7998.0,True,1800 Count 4 Piece Deep Pocket Bed Sheet Set -...,14510.0,True,New with tags,cozyarray,0.0
3,111669303786,219961.0,599.0,2999.0,True,Bedskirt Pins - Set of 8 Plastic Head Bed Skir...,4193.0,True,New with tags,cozyarray,0.0
4,111736675898,219960.0,599.0,2999.0,True,1800 Pillow Case Set Standard or King Ultra So...,27897.0,True,New with tags,cozyarray,0.0


In [20]:
#3(e)
warnings.filterwarnings("ignore")
SQL_DB = "msba"
TERM = "blog"

#Create msba database in mysql
SQL_TABLE = "deals"
SQL_TABLE_DEF = "(" + \
        "item_id CHAR(12) NOT NULL PRIMARY KEY" + \
        ",seller_score INT" + \
        ",item_price INT" + \
        ",list_price INT" + \
        ",has_list_price BOOLEAN" + \
        ",title VARCHAR(100)" + \
        ",number_sold INT" + \
        ",return_allowed BOOLEAN" + \
        ",condition_status VARCHAR(100)" + \
        ",seller_name VARCHAR(100)" + \
        ",shipping_price_variable_exists BOOLEAN" + \
        ")"

#connect to server
conn = pymysql.connect(host='localhost', user = 'root', password = '')
cursor = conn.cursor()

query = "CREATE DATABASE IF NOT EXISTS " + SQL_DB
print(query)
cursor.execute(query);

query = "CREATE TABLE IF NOT EXISTS " + SQL_DB + "." + SQL_TABLE + " " + SQL_TABLE_DEF + ";";
print(query)
cursor.execute(query);
cursor.close()
conn.close()

#Insert dataframe to sql
engine = create_engine('mysql://root:@localhost/msba')
items.to_sql('deals', con=engine, if_exists='replace')
engine.execute("SELECT * FROM deals").fetchall()

CREATE DATABASE IF NOT EXISTS msba
CREATE TABLE IF NOT EXISTS msba.deals (item_id CHAR(12) NOT NULL PRIMARY KEY,seller_score INT,item_price INT,list_price INT,has_list_price BOOLEAN,title VARCHAR(100),number_sold INT,return_allowed BOOLEAN,condition_status VARCHAR(100),seller_name VARCHAR(100),shipping_price_variable_exists BOOLEAN);


[(0, '110841365290', 219961.0, 799.0, 1698.0, 1, 'SET OF 1500 TC PILLOWCASES TWO PILLOW CASES PER SET ALL SIZES ALL COLORS', 2795.0, 1, 'New with tags', 'cozyarray', 0.0),
 (1, '110924201040', 219961.0, 799.0, 1798.0, 1, '1200 SERIES PILLOWCASES - 2 Pillow Cases Per Set. King Size Standard Size - SALE', 14867.0, 1, 'New with tags', 'cozyarray', 0.0),
 (2, '111634181825', 219961.0, 2199.0, 7998.0, 1, '1800 Count 4 Piece Deep Pocket Bed Sheet Set - Exclusive Spring Collection!!', 14510.0, 1, 'New with tags', 'cozyarray', 0.0),
 (3, '111669303786', 219961.0, 599.0, 2999.0, 1, 'Bedskirt Pins - Set of 8 Plastic Head Bed Skirt Dust Ruffle Holding Pins', 4193.0, 1, 'New with tags', 'cozyarray', 0.0),
 (4, '111736675898', 219960.0, 599.0, 2999.0, 1, '1800 Pillow Case Set Standard or King Ultra Soft Pillowcase Set of 2 Pillowcases', 27897.0, 1, 'New with tags', 'cozyarray', 0.0),
 (5, '112295879879', 219961.0, 799.0, 2999.0, 1, 'Bed Skirt Dust Ruffle Pins - Pack Of 16 Pins - Keeps Bedskirt In P

In [21]:
#3(f)
#Seller Score
print(items.groupby(['has_list_price','condition_status'])['seller_score'].mean())
print(items.groupby(['has_list_price','condition_status'])['seller_score'].min())
print(items.groupby(['has_list_price','condition_status'])['seller_score'].max())

has_list_price  condition_status
False           New                  37978.500000
                New with tags        60919.588889
True            New                  86692.727273
                New with tags       194988.293333
Name: seller_score, dtype: float64
has_list_price  condition_status
False           New                 20549.0
                New with tags         180.0
True            New                 65505.0
                New with tags        3198.0
Name: seller_score, dtype: float64
has_list_price  condition_status
False           New                  99661.0
                New with tags       420351.0
True            New                 147004.0
                New with tags       799196.0
Name: seller_score, dtype: float64


In [22]:
#Item Price
print(items.groupby(['has_list_price','condition_status'])['item_price'].mean()/100)
print(items.groupby(['has_list_price','condition_status'])['item_price'].min()/100)
print(items.groupby(['has_list_price','condition_status'])['item_price'].max()/100)

has_list_price  condition_status
False           New                 15.660000
                New with tags       27.651889
True            New                 24.103636
                New with tags       23.655067
Name: item_price, dtype: float64
has_list_price  condition_status
False           New                 5.50
                New with tags       5.99
True            New                 9.99
                New with tags       5.99
Name: item_price, dtype: float64
has_list_price  condition_status
False           New                  39.99
                New with tags       134.99
True            New                  94.99
                New with tags        79.81
Name: item_price, dtype: float64


In [23]:
#Shipping Price
print(items.groupby(['has_list_price','condition_status'])['shipping_price'].mean()/100)
print(items.groupby(['has_list_price','condition_status'])['shipping_price'].min()/100)
print(items.groupby(['has_list_price','condition_status'])['shipping_price'].max()/100)

has_list_price  condition_status
False           New                 0.0
                New with tags       0.0
True            New                 0.0
                New with tags       0.0
Name: shipping_price, dtype: float64
has_list_price  condition_status
False           New                 0.0
                New with tags       0.0
True            New                 0.0
                New with tags       0.0
Name: shipping_price, dtype: float64
has_list_price  condition_status
False           New                 0.0
                New with tags       0.0
True            New                 0.0
                New with tags       0.0
Name: shipping_price, dtype: float64


In [24]:
#Item number sold
print(items.groupby(['has_list_price','condition_status'])['number_sold'].mean())
print(items.groupby(['has_list_price','condition_status'])['number_sold'].min())
print(items.groupby(['has_list_price','condition_status'])['number_sold'].max())

has_list_price  condition_status
False           New                 1979.125000
                New with tags       1416.704545
True            New                 3774.100000
                New with tags       7720.100000
Name: number_sold, dtype: float64
has_list_price  condition_status
False           New                 59.0
                New with tags        1.0
True            New                 73.0
                New with tags        1.0
Name: number_sold, dtype: float64
has_list_price  condition_status
False           New                   6278.0
                New with tags         6980.0
True            New                  28143.0
                New with tags       181517.0
Name: number_sold, dtype: float64


In [ ]:
# 3(g)
# With a list price, the mean, min and max value of item sales are larger than sales of items without a list price
# So based on our observations, while there exist a list price, items are more likely to be sold. 
